In [1]:
import pyteal as pt

import ast
# from dataclasses import dataclass
import json
import inspect
# import dis
from typing import Iterable

In [2]:
hello = pt.Seq(pt.Log(pt.Concat(pt.Bytes("hello"),
    pt.Bytes("world"))), 
    pt.Int(1),
)

In [3]:
goodbye = pt.Return(pt.Int(42))

## EDGE CASE PROBLEM - Will not map operator in PyTeal code to TEAL when operator on its own line

This seems to be a systemic issue with `inspect.stack()` and frames. For example, consider the following cell and try to debug it.

You _CANNOT EVEN SET BREAKPOINTS_ at the `+` operators.

In [4]:
x = 42
y = (
    pt.Int(1) 
    + 
    pt.Int(2) 
    + 
    pt.Int(3) 
    + 
    pt.Int(4) 
    + 
    pt.Int(5)
)

In [5]:
from an_expression import goodnum
from some_subroutines import double_exp, cat
from pyteal.types import TealType

x = pt.ScratchVar(TealType.uint64)
program = {
    "pure": pt.Seq(
        pt.Pop(hello), 
        pt.Pop(goodnum),
        x.store(pt.Int(1000)),
        pt.Pop(pt.Int(1337) - pt.Int(42)),
        goodbye,
    ),
    "simple add": pt.Int(1) + pt.Int(2) + pt.Int(3) + pt.Int(4) + pt.Int(5),
    "line sep add": pt.Int(1) 
    +
    pt.Int(2)
     +
    pt.Int(3)
     +
    pt.Int(4)
     +
    pt.Int(5),
    "line sep pt.Add": pt.Add(
        pt.Add(
            pt.Add(
                pt.Add(
                    pt.Int(1),
                    pt.Int(2),
                ),
                pt.Int(3),
            ),
            pt.Int(4),
        ),
        pt.Int(5),
    ),
    "some subroutines":  pt.Seq(
        pt.Log(cat(pt.Bytes("Mapping "), pt.Bytes("PyTeal"))),
        pt.Log(pt.Bytes("this compiles but it'll blow up when executed")),
        double_exp(),
    )
}

In [6]:
i = 0

teals, lines, components = pt.compileTeal(program["some subroutines"], pt.Mode.Application, version=7)

In [7]:
print(f"""{teals=}
{lines=}
{components=}""")

teals='#pragma version 7\nbyte "Mapping "\nbyte "PyTeal"\ncallsub cat_2\nlog\nbyte "this compiles but it\'ll blow up when executed"\nlog\ncallsub doubleexp_1\nreturn\n\n// exp\nexp_0:\nint 2\nint 10\nexp\nretsub\n\n// double_exp\ndoubleexp_1:\ncallsub exp_0\ncallsub exp_0\nexp\nretsub\n\n// cat\ncat_2:\nstore 1\nstore 0\nload 0\nload 1\nconcat\nretsub'
lines=['#pragma version 7', 'byte "Mapping "', 'byte "PyTeal"', 'callsub cat_2', 'log', 'byte "this compiles but it\'ll blow up when executed"', 'log', 'callsub doubleexp_1', 'return', '\n// exp\nexp_0:', 'int 2', 'int 10', 'exp', 'retsub', '\n// double_exp\ndoubleexp_1:', 'callsub exp_0', 'callsub exp_0', 'exp', 'retsub', '\n// cat\ncat_2:', 'store 1', 'store 0', 'load 0', 'load 1', 'concat', 'retsub']
components=[TealOp(byte, '"Mapping "'), TealOp(byte, '"PyTeal"'), TealOp(callsub, 'cat_2'), TealOp(log), TealOp(byte, '"this compiles but it\'ll blow up when executed"'), TealOp(log), TealOp(callsub, 'doubleexp_1'), TealOp(return), TealLa

In [13]:
class PyTealFrame:
    def __init__(self, frame: inspect.FrameInfo | None):
        self.frame = frame
        self.source = None
        self.ast = None

    commentary = {
        "# T2PT1": "PyTeal generated label",
        "# T2PT2": "PyTeal generated return for TealType.none",
        "# T2PT3": "PyTeal generated return for non-null TealType",
        "# T2PT4": "PyTeal generated subroutine parameter handler instruction",
    }

    def location(self) -> str:
        return f"{self.file()}:{self.lineno()}" if self.frame else ""

    def file(self) -> str:
        return self.frame.filename if self.frame else ""

    def lineno(self) -> int | None:
        return self.frame.lineno if self.frame else None

    def code(self) -> str:
        raw = ("".join(self.frame.code_context)).strip() if self.frame and self.frame.code_context else ""
        if "# T2PT" not in raw:
            return raw
        for k, v in self.commentary.items():
            if k in raw:
                return f"{v}: {raw}"
        return f"Unhandled # T2PT commentary: {raw}"
        
    
    def __str__(self, verbose: bool = True) -> str:
        if not self.frame:
            return "None"

        spaces = "\n\t\t\t"
        short = f"<{self.code()}>{spaces}@{self.location()}"
        if not verbose:
            return short

        return f"""{short}
{self.source=}
{self.ast=}
{self.frame.index=}
{self.frame.function=}
{self.frame.frame=}"""

    def __repr__(self) -> str:
        return self.__str__(verbose=False)

    @classmethod
    def convert(cls, fs: inspect.FrameInfo | list):
        if isinstance(fs, inspect.FrameInfo):
            return cls(fs)
        return [cls.convert(f) for f in fs]

def get_source_map(
    teal_components: list[pt.TealOp],
    stackIndex: int = 2,
    firstLine: int = 2,
) -> dict:
    best_frames = []
    before, after = [], []
    for tc in teal_components:
        f, a, b = best_frame_and_more(tc)
        best_frames.append(f)
        before.append(b)
        after.append(a)

    return {i + firstLine: (t, best_frames[i], after[i], before[i]) for i, t in enumerate(teal_components)}


def best_frame_and_more(t: pt.TealComponent) -> tuple[inspect.FrameInfo | None, list[inspect.FrameInfo], list[inspect.FrameInfo]]:
    if not t.expr:
        assert isinstance(t, pt.TealLabel), f"expected TealLabel type but got {type(t)}"
        frames_reversed = t.frames
        path_match = "pyteal/ir"
    else:
        frames_reversed = t.expr.frames
        path_match = "pyteal/ast"

    pyteals = [
        path_match,
        "tests/abi_roundtrip.py",
        "tests/blackbox.py",
        "tests/compilea_asserts.py",
        "tests/mock_version.py",
    ]
    
    pyteal_idx = [any(w in f.filename for w in pyteals) for f in frames_reversed]

    in_pt, first_pt_entrancy = False, None
    for i, is_pyteal in enumerate(pyteal_idx):
        if is_pyteal and not in_pt:
            in_pt = True
            continue
        if not is_pyteal and in_pt:
            first_pt_entrancy = i
            break

    if first_pt_entrancy is None:
        return None, [], []
    return tuple(PyTealFrame.convert([
        frames_reversed[first_pt_entrancy], 
        [frames_reversed[i] for i in range(first_pt_entrancy-1, -1, -1)], 
        [frames_reversed[i] for i in range(first_pt_entrancy+1, len(frames_reversed))],
    ])) # type: ignore

In [14]:
m = get_source_map(components)

# Wrongs!!!
```
12: (TealOp(callsub, 'exp_0'),
      return self.subroutine(*args, **kwargs),
      [return self.subroutine.invoke(list(args)),
       return SubroutineCall(,
       args[0].frames = inspect.stack()],
      [return exp() ** exp()  # type: ignore,
       subroutine_body = subroutine.implementation(*loaded_args, **abi_output_kwargs),
```

In [15]:
from pprint import pprint
pprint(m, indent=1)

{2: (TealOp(byte, '"Mapping "'),
     <pt.Log(cat(pt.Bytes("Mapping "), pt.Bytes("PyTeal"))),>
			@/var/folders/xf/ks83gjdd16dctb70r8p79w8h0000gp/T/ipykernel_60295/4614554.py:38,
     [<return init(*args, **kwargs)>
			@/Users/zeph/github/tzaffi/pyteal/pyteal/ast/expr.py:16,
      <super().__init__()>
			@/Users/zeph/github/tzaffi/pyteal/pyteal/ast/bytes.py:45,
      <args[0].frames = inspect.stack()>
			@/Users/zeph/github/tzaffi/pyteal/pyteal/ast/expr.py:14],
     [<exec(code_obj, self.user_global_ns, self.user_ns)>
			@/Users/zeph/github/tzaffi/pyteal/py310ptt/lib/python3.10/site-packages/IPython/core/interactiveshell.py:3398,
      <if await self.run_code(code, result, async_=asy):>
			@/Users/zeph/github/tzaffi/pyteal/py310ptt/lib/python3.10/site-packages/IPython/core/interactiveshell.py:3338,
      <has_raised = await self.run_ast_nodes(code_ast.body, cell_name,>
			@/Users/zeph/github/tzaffi/pyteal/py310ptt/lib/python3.10/site-packages/IPython/core/interactiveshell.py:3135,
    

In [11]:
def add_asts_to_frames(frames: Iterable[PyTealFrame]):
    """And do it without compiling more than once per file"""
    def get_jupyter_index(code) -> int | None:
        indices = [i for i, cell_code in enumerate(_ih) if code in cell_code]
        return indices[0] if indices else None


    locations = [f.location() for f in frames]
    files = [f.file() for f in frames]
    codes = [f.code() for f in frames]
    jupyter_idx = [get_jupyter_index(c) for c in codes]
    # file2source = {f. for i, idx in enumerate(jupyter_idx) }
    # sources = [_ih[idx] if idx is not None else open(files[i]).read() for i, idx in enumerate(jupyter_idx)]

    file2source = {}
    file2ast = {}
    for i, f in enumerate(files):
        if f in file2source:
            continue

        idx = jupyter_idx[i]
        source = _ih[idx] if idx is not None else open(f).read()

        file2source[f] = source
        file2ast[f] = ast.parse(source)

    
    for i, f in enumerate(frames):
        idx = jupyter_idx[i]
        file = f.file()
        f.source = file2source[file]
        f.ast = file2ast[file]
    
    
    return file2source, file2ast

In [12]:
frames = [pt_frame for (_, (_, pt_frame)) in m.items()]

add_asts_to_frames(frames)

ValueError: too many values to unpack (expected 2)

In [ ]:
import ast
from pprint import pprint

class CallVisitor(ast.NodeVisitor):
    def print(self, node):
        nl = "_" * 10 + "\n"
        print(f"{nl}Node{type(node)} [{node.lineno}:{node.col_offset}-->{node.end_lineno}:{node.end_col_offset}]")
        for f in node._fields:
            print(f"{f}={getattr(node, f)}")
        

    def visit_Call(self, node):
        self.print(node)
        ast.NodeVisitor.generic_visit(self, node)

    def visit_Name(self,node):
        self.print(node)
        # print('Node type: Name\nFields: ', node._fields)
        ast.NodeVisitor.generic_visit(self, node)

    def visit_Constant(self,node):
        self.print(node)
        # print('Node type: Constant\nFields: ', node._fields)
        ast.NodeVisitor.generic_visit(self, node)

    def visit_keyword(self,node):
        self.print(node)
        # print('Node type: keyword\nFields: ', node._fields)
        ast.NodeVisitor.generic_visit(self, node)



visitor = CallVisitor()
tree = ast.parse("myfunc(a, 100, found=True)", mode='eval')
pprint(ast.dump(tree))
visitor.visit(tree)

In [ ]:
visitor.visit(frames[1].ast)

In [ ]:
print(ast.dump(frames[1].ast, include_attributes=True, indent="  "))

In [ ]:
t = (1,2,3,4)

t.count(3)

In [ ]:
idx = 5
m[idx][1].lineno, m[idx][1].code_context, m[idx][1].frame, m[idx][1]._asdict()

In [ ]:
idx = 6
m[idx][1].lineno, m[idx][1].code_context, m[idx][1].frame, m[idx][1]._asdict()

In [ ]:
idx = 7
m[idx][1].lineno, m[idx][1].code_context, m[idx][1].frame, m[idx][1]._asdict()

In [ ]:
len(m)

In [ ]:
m.keys()